In [ ]:
import pandas as pd

###################################################
# Loading datasets
##################################################

account_df = pd.read_csv('raw_file/account-statement-1-1-2024-12-31-2024.csv', delimiter=';')

# remove empty columns
if '' in account_df.columns:
    account_df = account_df.drop('', axis=1)

symbols_df = pd.read_csv('raw_file/symbols.csv', delimiter=';')
country_df = pd.read_csv('raw_file/country.csv')

# Missing symbols (because they are not in the symbols file) but they are in the account file
unique_symbols_in_transactions = set(account_df['Symbol'].unique())
unique_symbols_in_lookup = set(symbols_df['symbol'].unique())
missing_symbols = unique_symbols_in_transactions - unique_symbols_in_lookup

if missing_symbols:
    missing_symbols_list = [s for s in missing_symbols if pd.notna(s)]
    if missing_symbols_list:
        # Remove rows with missing values
        account_df = account_df[~account_df['Symbol'].isin(missing_symbols_list)]
        
        # Update the unique symbols list after filtering
        unique_symbols_in_transactions = set(account_df['Symbol'].unique())
        missing_symbols = unique_symbols_in_transactions - unique_symbols_in_lookup

# Check for dividend transactions (handle both spellings)
dividend_variations = ['DIVIDEND', 'DIVIDENT']
dividend_transactions = account_df[account_df['TransactionType'].isin(dividend_variations)]


###################################################
# Part 2: Create Dimension Tables
##################################################

# Dim_Time -  (Quarter only)

account_df['Date'] = pd.to_datetime(account_df['Date'], format='%d/%m/%Y %H:%M:%S')

# Extract only quarter
account_df['quarter'] = 'Q' + account_df['Date'].dt.quarter.astype(str)

# Create unique quarters only
unique_quarters = account_df[['quarter']].drop_duplicates()

dim_time = unique_quarters.copy()
dim_time = dim_time.sort_values(['quarter']).reset_index(drop=True)
dim_time['time_id'] = range(1, len(dim_time) + 1)

# Keep only the essential columns: time_id, quarter
dim_time = dim_time[['time_id', 'quarter']]


# Dim_Geography 

# Country name mapping to handle mismatches between country names in symbols data
country_name_mapping = {
    'Taiwan': 'Taiwan, Province of China',  # symbols file has 'Taiwan', country file has 'Taiwan, Province of China'
    'Turkey': 'Türkiye',  # symbols file has 'Turkey', country file has 'Türkiye'
}

# Apply country mapping to symbols data
symbols_df_mapped = symbols_df.copy()
symbols_df_mapped['country_mapped'] = symbols_df_mapped['country'].map(country_name_mapping).fillna(symbols_df_mapped['country'])

# Get unique countries (using the mapped names)
unique_geography = symbols_df_mapped[['country_mapped']].drop_duplicates()

dim_geography = unique_geography.copy()
dim_geography.columns = ['country_name']

# Remove rows with null values 
dim_geography = dim_geography.dropna().drop_duplicates().reset_index(drop=True)
dim_geography = dim_geography.sort_values(['country_name']).reset_index(drop=True)

# Add geography_id
dim_geography['geography_id'] = range(1, len(dim_geography) + 1)

# Dim_Symbol

dim_symbol = symbols_df[['symbol', 'company_name', 'sector', 'industry']].copy()
dim_symbol = dim_symbol.sort_values('symbol').reset_index(drop=True)
dim_symbol['symbol_id'] = range(1, len(dim_symbol) + 1)

# Dim_TransactionType

unique_types = sorted(account_df['TransactionType'].unique())
dim_transaction_type = pd.DataFrame({
    'transaction_type': unique_types,
    'transaction_type_id': range(1, len(unique_types) + 1)
})

###################################################
# Part 3: Create Fact Table
##################################################

fact_transactions = account_df.copy()

# Ensure date_only is datetime type for proper merging
fact_transactions['quarter_for_merge'] = 'Q' + fact_transactions['Date'].dt.quarter.astype(str)

# Add foreign keys
# Time dimension - join on quarter
fact_transactions = fact_transactions.merge(
    dim_time[['quarter', 'time_id']], 
    left_on='quarter_for_merge', 
    right_on='quarter', 
    how='inner'
)

# Symbol dimension
fact_transactions = fact_transactions.merge(
    dim_symbol[['symbol', 'symbol_id']], 
    left_on='Symbol', 
    right_on='symbol', 
    how='inner'
)

# Geography dimension 
symbol_geography = symbols_df[['symbol', 'country']].copy()

# Apply country mapping
symbol_geography['country_mapped'] = symbol_geography['country'].map(country_name_mapping).fillna(symbol_geography['country'])

symbol_geography = symbol_geography.merge(
    dim_geography[['country_name', 'geography_id']], 
    left_on='country_mapped', 
    right_on='country_name', 
    how='inner'
)

fact_transactions = fact_transactions.merge(
    symbol_geography[['symbol', 'geography_id']], 
    on='symbol', 
    how='inner'
)

# Transaction type dimension
fact_transactions = fact_transactions.merge(
    dim_transaction_type, 
    left_on='TransactionType', 
    right_on='transaction_type', 
    how='inner'
)

# Final fact table 
fact_transactions_final = fact_transactions[[
    'IDTransaction', 'time_id', 'geography_id', 'symbol_id', 
    'transaction_type_id'
]].copy()
fact_transactions_final.columns = [
    'transaction_id', 'time_id', 'geography_id', 'symbol_id', 
    'transaction_type_id'
]

# Convert data types handle problem to microsft lida
fact_transactions_final['transaction_id'] = fact_transactions_final['transaction_id'].astype('int64')
fact_transactions_final['time_id'] = fact_transactions_final['time_id'].astype('int32')
fact_transactions_final['geography_id'] = fact_transactions_final['geography_id'].astype('int32')
fact_transactions_final['symbol_id'] = fact_transactions_final['symbol_id'].astype('int32')
fact_transactions_final['transaction_type_id'] = fact_transactions_final['transaction_type_id'].astype('int32')



###################################################
# Create Merged Dataset for Streamlit
##################################################

# Create complete merged dataset by joining fact table with all dimensions
merged_data = fact_transactions_final.copy()

# Join with time dimension
merged_data = merged_data.merge(dim_time, on='time_id', how='left')

# Join with geography dimension
merged_data = merged_data.merge(dim_geography, on='geography_id', how='left')

# Join with symbol dimension
merged_data = merged_data.merge(dim_symbol, on='symbol_id', how='left')

# Join with transaction type dimension
merged_data = merged_data.merge(dim_transaction_type, on='transaction_type_id', how='left')

# Select only the columns needed for analysis 
merged_final = merged_data[[
    'quarter', 'country_name', 'symbol', 'company_name', 
    'sector', 'industry', 'transaction_type'
]].copy()



###################################################
# Save data
##################################################
'''
# Save dimension tables (for reference)
dim_time.to_csv('dim_time.csv', index=False)
dim_geography.to_csv('dim_geography.csv', index=False)
dim_symbol.to_csv('dim_symbol.csv', index=False)
dim_transaction_type.to_csv('dim_transaction_type.csv', index=False)

# Save fact table (for reference)
fact_transactions_final.to_csv('fact_transactions.csv', index=False)
'''
# Save merged dataset for Streamlit 
merged_final.to_csv('transactions_merged.csv', index=False)